In [2]:
from __future__ import absolute_import, division, print_function
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import multiprocessing
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) #lets log everything 

In [8]:
nltk.download("punkt") #splitting text in to sentences
nltk.download("stopwords") # getting all un neccessary words link ,'and' 'or' out of text

[nltk_data] Downloading package punkt to
[nltk_data]     /home/codejunky237/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codejunky237/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
book_filenames = sorted(glob.glob("/home/codejunky237/ml/word2vec/word_vectors_game_of_thrones-LIVE/data/*.txt"))
print(type(book_filenames))
print(len(book_filenames))




<type 'list'>
5


In [10]:
# create a corpus of the books
word_corpus = u""
for book_filename in book_filenames:
    print("what is this format '{0}'".format(book_filename))
    with codecs.open(book_filename, "r","utf-8") as bk: # open books with utf-8 format
        word_corpus += bk.read()
    print("Corpus is now {0} characters long".format(len(word_corpus)))
    print(type(word_corpus))
    


what is this format '/home/codejunky237/ml/word2vec/word_vectors_game_of_thrones-LIVE/data/got1.txt'
Corpus is now 1770659 characters long
<type 'unicode'>
what is this format '/home/codejunky237/ml/word2vec/word_vectors_game_of_thrones-LIVE/data/got2.txt'
Corpus is now 4071041 characters long
<type 'unicode'>
what is this format '/home/codejunky237/ml/word2vec/word_vectors_game_of_thrones-LIVE/data/got3.txt'
Corpus is now 6391405 characters long
<type 'unicode'>
what is this format '/home/codejunky237/ml/word2vec/word_vectors_game_of_thrones-LIVE/data/got4.txt'
Corpus is now 8107945 characters long
<type 'unicode'>
what is this format '/home/codejunky237/ml/word2vec/word_vectors_game_of_thrones-LIVE/data/got5.txt'
Corpus is now 9719485 characters long
<type 'unicode'>


In [11]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') #get sentences from text in unicode format
raw_words = tokenizer.tokenize(word_corpus)
print(raw_words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [12]:
#clean the data
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

sentence_list = []

for sentence in raw_words:
    if len(sentence) > 0 :
        sentence_list.append(sentence_to_wordlist(sentence))
print(len(sentence_list))
print(raw_words[0])
print(sentence_list[0])

128868
This edition contains the complete text of the original hardcover edition.
[u'This', u'edition', u'contains', u'the', u'complete', u'text', u'of', u'the', u'original', u'hardcover', u'edition']


In [13]:
token_count = sum([len(sentence) for sentence in sentence_list])
print("the token coun '{0}'".format(token_count))

the token coun '1818103'


In [14]:
# this step we build the model
# dimensionality of the resulting vector 
num_features = 300 # less dimensionality easy to train 
min_word_count = 3
num_workers = multiprocessing.cpu_count() #number of workers 

context_window = 7 # number of surrounding names for each selected word
down_sampling = 1e-3 #reducing sample size
seed = 1
thrones2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_window,
    sample=down_sampling
)


In [15]:
thrones2vec.build_vocab(sentence_list)

2018-06-12 13:55:34,834 : INFO : collecting all words and their counts
2018-06-12 13:55:34,837 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-12 13:55:34,908 : INFO : PROGRESS: at sentence #10000, processed 140984 words, keeping 10280 word types
2018-06-12 13:55:34,976 : INFO : PROGRESS: at sentence #20000, processed 279730 words, keeping 13558 word types
2018-06-12 13:55:35,049 : INFO : PROGRESS: at sentence #30000, processed 420336 words, keeping 16598 word types
2018-06-12 13:55:35,113 : INFO : PROGRESS: at sentence #40000, processed 556581 words, keeping 18324 word types
2018-06-12 13:55:35,175 : INFO : PROGRESS: at sentence #50000, processed 686247 words, keeping 19714 word types
2018-06-12 13:55:35,237 : INFO : PROGRESS: at sentence #60000, processed 828497 words, keeping 21672 word types
2018-06-12 13:55:35,299 : INFO : PROGRESS: at sentence #70000, processed 973830 words, keeping 23093 word types
2018-06-12 13:55:35,361 : INFO : PROGRESS: at 

In [18]:

print("Word2Vec vocabulary length:", len(thrones2vec.wv.vocab))

Word2Vec vocabulary length: 17277


In [23]:
thrones2vec.train(sentence_list,total_examples = thrones2vec.corpus_count,epochs=10)

2018-06-12 14:32:42,982 : INFO : training model with 4 workers on 17277 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=7
2018-06-12 14:32:44,234 : INFO : EPOCH 1 - PROGRESS: at 5.41% examples, 76456 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:32:45,273 : INFO : EPOCH 1 - PROGRESS: at 13.30% examples, 90599 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:32:46,326 : INFO : EPOCH 1 - PROGRESS: at 21.50% examples, 96955 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:32:47,335 : INFO : EPOCH 1 - PROGRESS: at 30.03% examples, 101422 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:32:48,344 : INFO : EPOCH 1 - PROGRESS: at 39.54% examples, 105607 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:32:49,350 : INFO : EPOCH 1 - PROGRESS: at 46.57% examples, 104612 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:32:50,403 : INFO : EPOCH 1 - PROGRESS: at 55.13% examples, 106510 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:32:51,435 : INFO : EPOCH 1 - PROGRESS: at 63.32% 

2018-06-12 14:33:43,663 : INFO : EPOCH 5 - PROGRESS: at 70.29% examples, 102581 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:33:44,712 : INFO : EPOCH 5 - PROGRESS: at 77.37% examples, 101159 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:33:45,712 : INFO : EPOCH 5 - PROGRESS: at 84.49% examples, 100409 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:33:46,754 : INFO : EPOCH 5 - PROGRESS: at 89.98% examples, 98918 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:33:47,765 : INFO : EPOCH 5 - PROGRESS: at 96.47% examples, 98801 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:33:48,078 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-06-12 14:33:48,140 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-06-12 14:33:48,165 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-06-12 14:33:48,239 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-06-12 14:33:48,240 : INFO : EPOCH - 5 : training on 1818103 raw word

2018-06-12 14:34:40,698 : INFO : EPOCH - 9 : training on 1818103 raw words (1404765 effective words) took 13.1s, 106899 effective words/s
2018-06-12 14:34:41,727 : INFO : EPOCH 10 - PROGRESS: at 5.45% examples, 75621 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:34:42,763 : INFO : EPOCH 10 - PROGRESS: at 12.64% examples, 86288 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:34:43,808 : INFO : EPOCH 10 - PROGRESS: at 20.92% examples, 94396 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:34:44,856 : INFO : EPOCH 10 - PROGRESS: at 28.24% examples, 94836 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:34:45,949 : INFO : EPOCH 10 - PROGRESS: at 35.36% examples, 92803 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:34:46,978 : INFO : EPOCH 10 - PROGRESS: at 43.14% examples, 93565 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:34:48,000 : INFO : EPOCH 10 - PROGRESS: at 50.87% examples, 96367 words/s, in_qsize 8, out_qsize 0
2018-06-12 14:34:49,033 : INFO : EPOCH 10 - PROGRESS: at 58.34% examples

(14046272, 18181030)

In [24]:
thrones2vec.most_similar("Stark")

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2018-06-12 14:36:37,800 : INFO : precomputing L2-norms of word weight vectors


[(u'Eddard', 0.6736850738525391),
 (u'executed', 0.5348050594329834),
 (u'divulge', 0.507888913154602),
 (u'ROSLIN', 0.4971054792404175),
 (u'Winterfell', 0.49284863471984863),
 (u'Knelt', 0.48720240592956543),
 (u'Edrick', 0.485209584236145),
 (u'Burner', 0.48321765661239624),
 (u'absently', 0.482699453830719),
 (u'Snowbeard', 0.48158159852027893)]

In [28]:
thrones2vec.most_similar("Jon")

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'Snow', 0.6602096557617188),
 (u'Ygritte', 0.5214747786521912),
 (u'Qhorin', 0.5154229402542114),
 (u'Stonesnake', 0.4834851026535034),
 (u'Orell', 0.46222802996635437),
 (u'Benjen', 0.4532366991043091),
 (u'Val', 0.4521523118019104),
 (u'Ghost', 0.45110154151916504),
 (u'cackling', 0.45073816180229187),
 (u'Coldhands', 0.45026370882987976)]